#  set autocomplete

In [1]:
%config IPCompleter.greedy=True

# Eager Mode Mnist

In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import BatchNormalization, Flatten, Dense
import numpy as np

mnist image : 28,28 

In [3]:
mnist = tf.keras.datasets.mnist

# Create Model<br>
mnist 분류기 모델을 정의 <br>
conv2d<br>
activation relu <br>
batch normalize <br>
conv2d<br>
activation relu<br>
batch normalize<br>
flatten<br>
dense(10)<br>

In [4]:
class Mnist_with_BN(tf.keras.Model):
    def __init__(self):
        super(Mnist_with_BN, self).__init__()
        self.conv1 = Conv2D(16,[3,3], activation='relu')
        self.bn1 = BatchNormalization()
        self.conv2 = Conv2D(16,[3,3], activation='relu')
        self.bn2 = BatchNormalization()
        self.conv3 = Conv2D(16,[3,3], activation='relu')
        self.bn3 = BatchNormalization()
        self.flat = Flatten()
        self.dense =  Dense(10, activation="softmax")
    
    def __call__(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.flat(x)
        x = self.dense(x)
        return x
    
model = Mnist_with_BN()

# Load Data <br>
데이터를 로딩함.

In [5]:
# Fetch and format the mnist data
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255.0, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

이미지 차원 및 결과물 차원 확인

In [6]:
# for images, labels in dataset.take(1):
#     print(images[0:1].shape)
#     print("Logis", model(images[0:1]).shape)

# Define Optimizer

AdamOptimizer를 사용함

In [7]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# Train

In [8]:
@tf.function
def train_step(model, image, label):
  with tf.GradientTape() as tape:
    predictions = model(image)
    loss = loss_object(label, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(label, predictions)
  #print(f"loss : {loss.result()}, acc : {train_accuracy.result()*100}")
    

In [13]:
#device = '/cpu:0' if tf.num_gpus() == 0 else '/gpu:0'
#print('using device %s' %device)
epoch=2
loss_history = []
acc_history = []
   
for _ in range(epoch):
    for images, labels in dataset:
        train_step(model, images, labels)
    print(f"epoch:{epoch}, loss:{train_loss.result()}, acc:{train_accuracy.result()*100}")    
             

epoch:2, loss:0.07473435997962952, acc:97.76944732666016
epoch:2, loss:0.062029145658016205, acc:98.1379165649414
